In [36]:
import gymnasium as gym
import numpy as np

# Resolvendo o ambiente Cliff Walking com QLearning tabular

O [Cliff Walking](https://gymnasium.farama.org/environments/toy_text/cliff_walking/) é um dos mais simples ambientes de aprendizado por reforço (RL), sendo ideal para entender o funcionamento do QLearning tabular, um dos algoritmos mais fundamentais para iniciantes. A ideia do Cliff Walking é simples: o agente deve ser capaz de atravessar um tabuleiro do início ao fim tomando cuidado para não cair em um penhasco. Se o agente cair no penhasco, ele retorna para o início do tabuleiro e leva uma penalidade de recompensa.

<img src="media/cliff_walking.gif" width="200">

Abaixo seguem algumas informações importantes para a modelagem do ambiente como um Processo de Decisão de Markov (MDP):

### Espaço de ações

O espaço de ações é discreto e contém os inteiros do intervalo {0, 3}. Uma ação deve indicar a direção de um movimento:
* 0: Cima
* 1: Direita
* 2: Baixo
* 3: Esquerda

### Espaço de estados

O estado representa a posição do jogador no tabuleiro. Logo, o espaço de estados também é discreto e contém os inteiros do intervalo {0, 47}. O valor numérico da posição do agente no tabuleiro pode ser obtido como linha_atual * nlinhas + coluna, sendo que as linhas e colunas começam em 0.



In [37]:
cliff_walking = gym.make(
    "CliffWalking-v0",
    # render_mode = 'human' # caso queira acompanhar o processo de treinamento visualmente, descomente essa linha
)

### Criando a tabela de QValores

A tabela de Qvalores recebe como entrada um par (estado, ação) e deve retornar o QValor desse par. Um QValor pode ser interpretado como "a recompensa acumulada total esperada por executar a ação A no estado S e depois seguir a mesma política até o final do episódio". Logo, devemos ter uma linha da tabela para cada um dos 48 estados, sendo que cada linha deve ter uma coluna para cada uma das 4 ações. Além disso, os QValores devem possuir um valor inicial que, nesse caso, será zero.

<figure>
    <img src="media/qtable.png" alt="sample qtable" width="400">
    <figcaption> Exemplo de tabela de Qvalores. Fonte: <a href="https://www.datacamp.com/tutorial/introduction-q-learning-beginner-tutorial"> Datacamp </a> </figcaption>
</figure>

In [38]:
def new_q_table(n_states, n_actions):
    return np.zeros((n_states, n_actions))

### Amostrando ações com a política $\epsilon$-greedy

No final do treinamento, espera-se que a melhor ação para cada estado seja aquela cujo QValor é o maior. No entanto, para que o QLearn convirja adequadamente, é necessário que no início do treinamento o agente "explore" bem o ambiente. Isto é, que ele visite um grande número de estados mesmo que não sejam necessariamente ótimos. Uma técnica amplamente utilizada para essa finalidade é a política $\epsilon$-greedy. Ela consiste em forçar o agente a escolher ações aleatoriamente com uma frequência que diminui conforme o treinamento avança.

In [39]:
def get_action(q_table, state, epsilon):
    if np.random.random() < epsilon:
        return np.random.randint(0, q_table.shape[1])
    return np.argmax(q_table[state])

### Atualização dos QValores utilizando a equação de Bellman

A cada passo do treinamento, o agente executará uma ação e utilizará a informação retornada pelo ambiente para atualizar os seus QValores e, assim, aprender a tabela ótima. A atualização dos seus QValores é realizada através da equação de Bellman:

$$Q_{t+1}(s_t, a_t) = (\alpha - 1) Q_t (s_t, a_t) + \alpha (R_{t+1} + \gamma \max_{a} Q_t (s_{t+1}, a))$$

In [40]:
def update_q_table(q_table, state, action, reward, next_state, terminated, alpha, gamma):
    if terminated:
        q_table[next_state] = np.zeros(q_table.shape[1]) # a recompensa esperada para o estado terminal é 0

    q_table[state, action] = (alpha - 1) * q_table[state, action] + alpha * (reward + gamma * np.max(q_table[next_state]))
    return q_table

### Loop de treinamento

No loop de treinamento, juntaremos todas as funções desenvolvidas até o momento. A ideia principal é definir um número máximo de episódios (estágio inicial até o estágio final) para que o agente colete experiências do ambiente e otimize sua tabela de QValores.

In [41]:
def train(env: gym.Env, 
          q_table,
          n_episodes=3000, 
          epsilon=0.9, 
          epsilon_decay=0.999, 
          alpha=0.1, 
          gamma=0.99):
     
    for episode in range(n_episodes):
        state, _ = env.reset()
        total_reward = 0
        step = 0
        done = False

        if epsilon > 0.001:
            epsilon *= epsilon_decay
        
        while not done and step < 1000: # 1000 steps max
            action = get_action(q_table, state, epsilon)
            next_state, reward, terminated, truncated, _ = env.step(action)
            q_table = update_q_table(q_table, state, action, reward, next_state, terminated, alpha, gamma)
            state = next_state
            total_reward += reward

            done = terminated or truncated

        if episode % 100 == 0:
            print(f"Episode {episode} - Total reward: {total_reward} - epsilon: {epsilon}")

    return q_table

### Treinando

In [42]:
q_table = new_q_table(n_states=48, n_actions=4)
trained_q_table = train(cliff_walking, q_table)

Episode 0 - Total reward: -61469 - epsilon: 0.8991
Episode 100 - Total reward: -10653 - epsilon: 0.8134986194699355
Episode 200 - Total reward: -4311 - epsilon: 0.7360471625842407
Episode 300 - Total reward: -2899 - epsilon: 0.6659696926115485
Episode 400 - Total reward: -383 - epsilon: 0.6025641480906593
Episode 500 - Total reward: -725 - epsilon: 0.545195309324691
Episode 600 - Total reward: -565 - epsilon: 0.49328843452021
Episode 700 - Total reward: -260 - epsilon: 0.44632350181590114
Episode 800 - Total reward: -332 - epsilon: 0.4038299995153185
Episode 900 - Total reward: -221 - epsilon: 0.3653822123303929
Episode 1000 - Total reward: -20 - epsilon: 0.33059495641157327
Episode 1100 - Total reward: -359 - epsilon: 0.29911972043659035
Episode 1200 - Total reward: -124 - epsilon: 0.27064117409787486
Episode 1300 - Total reward: -469 - epsilon: 0.24487400900939155
Episode 1400 - Total reward: -13 - epsilon: 0.22156008038394912
Episode 1500 - Total reward: -13 - epsilon: 0.20046582084

### Testando o agente